In [4]:
import numpy as np
from sklearn.mixture import GaussianMixture
import librosa
from sklearn.decomposition import PCA
import os

In [10]:
def getMfcc(wav):
        mfcc = librosa.feature.mfcc(y=wav,
                                    sr=16000,
                                    # n_fft=400,
                                    # hop_length=32,
                                    # n_mfcc=40
                                    )


        # min_val = np.min(mfcc)
        # max_val = np.max(mfcc)
        # mfcc = (mfcc - min_val) / (max_val - min_val)


        n_components = 1  # 축소할 차원 수 지정
        pca = PCA(n_components=n_components)


        mfcc = pca.fit_transform(mfcc)

        min_val = np.min(mfcc)
        max_val = np.max(mfcc)
        mfcc = (mfcc - min_val) / (max_val - min_val)

        return mfcc.T

In [11]:
# 음성 데이터 폴더 경로
train_data_file = 'fmcc_train.ctl'
test_data_file = 'fmcc_test.ctl'
test_label_file = 'fmcc_test_ref.txt'

# gmm = GaussianMixture(n_components=2, random_state=42, max_iter=10)

In [12]:
X_train = []
X_test = []
test_labels = []

# Train 데이터
with open("data/" + train_data_file, 'r') as file:
    for file_path in file:
        file_path = file_path.rstrip("\n")
        wav, frame = librosa.load("data/wav16k/train/" + file_path + ".wav", sr=16000, mono=True)
        mfcc = getMfcc(wav)
        X_train.append(mfcc)



# Test 데이터
with open("data/" + test_data_file, 'r') as file:
    for file_path in file:
        file_path = file_path.rstrip("\n")
        wav, frame = librosa.load("data/wav16k/test/" + file_path + ".wav", sr=16000, mono=True)
        mfcc = getMfcc(wav)
        X_test.append(mfcc)

# Test 데이터의 정답 레이블을 읽어와 저장
with open("data/" + test_label_file, 'r') as file:
    for i, txt in enumerate(file):
        _, labels = txt.rstrip("\n").split()

        if labels == "feml":
            labels = 0
        else: labels = 1

        test_labels.append(labels)

In [13]:
# from sklearn.utils.testing import ignore_warnings
import warnings
from sklearn.exceptions import ConvergenceWarning

# 데이터를 NumPy 배열로 변환
X_train = np.concatenate(X_train, axis=0)
X_test = np.concatenate(X_test, axis=0)
test_labels = np.array(test_labels)

In [14]:
# GMM 모델 학습
with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        model = GaussianMixture(n_components=2, init_params='random', random_state=0, tol=1e-9, max_iter=20)
        model.fit(X_train)

# Test 데이터셋에 대한 예측
predictions = model.predict(X_test)

# 정답 맞춘 비율 계산
accuracy = np.mean(predictions == test_labels.reshape(-1))
# print(predictions)
# print(test_labels.reshape(-1))

# 결과 출력
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 52.33%
